In [ ]:
from skyfield.api import load

# Load planetary data and time
planets = load('de421.bsp')
ts = load.timescale()
t = ts.now()

# Get Earth and Mars
earth = planets['earth']
mars = planets['mars']

# Observe Mars from Earth
astrometric = earth.at(t).observe(mars)
ra, dec, distance = astrometric.radec()

# Print results
print(f"Mars Right Ascension: {ra}")
print(f"Mars Declination: {dec}")
print(f"Distance (AU): {distance}")


In [ ]:
import sys
from PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout, QLabel, QComboBox
from PyQt5.QtCore import QTimer
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.figure import Figure
from skyfield.api import load

# Load planetary data
planets = load('de421.bsp')
ts = load.timescale()

# Only include planets actually in the kernel
planet_names = {
    'Mercury': 'mercury',
    'Venus': 'venus',
    'Earth': 'earth',
    'Mars': 'mars',
    'Jupiter': 'jupiter barycenter',
    'Saturn': 'saturn barycenter',
    'Uranus': 'uranus barycenter',
    'Neptune': 'neptune barycenter',
    'Pluto': 'pluto barycenter'
}

class PlanetTracker(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Real-Time Planet Tracker")
        self.setGeometry(100, 100, 600, 600)

        layout = QVBoxLayout()

        # Dropdown to select planet
        self.dropdown = QComboBox()
        self.dropdown.addItems(planet_names.keys())
        self.dropdown.currentIndexChanged.connect(self.update_data)

        # Info labels
        self.ra_label = QLabel("Right Ascension: ")
        self.dec_label = QLabel("Declination: ")
        self.dist_label = QLabel("Distance (AU): ")

        # Matplotlib plot
        self.figure = Figure(figsize=(5, 5))
        self.canvas = FigureCanvas(self.figure)
        self.ax = self.figure.add_subplot(111)
        self.ax.set_title("Planet Position (relative)")
        self.ax.set_xlim(-2, 2)
        self.ax.set_ylim(-2, 2)
        self.ax.grid(True)

        layout.addWidget(self.dropdown)
        layout.addWidget(self.ra_label)
        layout.addWidget(self.dec_label)
        layout.addWidget(self.dist_label)
        layout.addWidget(self.canvas)

        self.setLayout(layout)

        # Timer for updates
        self.timer = QTimer()
        self.timer.timeout.connect(self.update_data)
        self.timer.start(1000)

        self.update_data()

    def update_data(self):
        planet_name = self.dropdown.currentText()
        skyfield_name = planet_names[planet_name]
        t = ts.now()
        earth = planets['earth']
        target = planets[skyfield_name]

        astrometric = earth.at(t).observe(target)
        ra, dec, distance = astrometric.radec()
        pos = astrometric.position.au  # x, y, z in AU

        # Update labels
        self.ra_label.setText(f"Right Ascension: {ra}")
        self.dec_label.setText(f"Declination: {dec}")
        self.dist_label.setText(f"Distance (AU): {distance.au:.4f}")

        # Update plot
        self.ax.clear()
        self.ax.set_title("Planet Position (relative to Earth)")

        # Calculate limit based on distance with some margin
        limit = max(2, abs(pos[0]) * 1.2, abs(pos[1]) * 1.2)

        self.ax.set_xlim(-limit, limit)
        self.ax.set_ylim(-limit, limit)

        self.ax.grid(True)
        self.ax.plot(0, 0, 'bo', label='Earth')
        self.ax.plot(pos[0], pos[1], 'ro', label=planet_name)
        self.ax.legend()
        self.canvas.draw()
        
if __name__ == "__main__":
    app = QApplication(sys.argv)
    tracker = PlanetTracker()
    tracker.show()
    sys.exit(app.exec_())


In [ ]:
import sys
from PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout, QLabel, QComboBox
from PyQt5.QtCore import QTimer
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.figure import Figure
from mpl_toolkits.mplot3d import Axes3D
from skyfield.api import load

# Load planetary data
planets = load('de421.bsp')
ts = load.timescale()

# Added 'sun' here too
planet_names = {
    'Sun': 'sun',
    'Mercury': 'mercury',
    'Venus': 'venus',
    'Earth': 'earth',
    'Mars': 'mars',
    'Jupiter': 'jupiter barycenter',
    'Saturn': 'saturn barycenter',
    'Uranus': 'uranus barycenter',
    'Neptune': 'neptune barycenter',
    'Pluto': 'pluto barycenter'
}

def scale_position(pos):
    """
    Custom scaling to spread inner planets apart for better visualization.
    Applies a simple function that grows faster for smaller distances.
    """
    import numpy as np
    # Example: amplify small distances more than big distances
    scale_factor = 2
    scaled = []
    for x in pos:
        sign = 1 if x >= 0 else -1
        # Apply sqrt scaling preserving sign (to spread inner planets out)
        scaled.append(sign * (abs(x) ** 0.5) * scale_factor)
    return scaled

class PlanetTracker3D(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("3D Real-Time Planet Tracker with Sun")
        self.setGeometry(100, 100, 700, 700)

        layout = QVBoxLayout()

        self.dropdown = QComboBox()
        self.dropdown.addItems(['All Planets'] + list(planet_names.keys()))
        self.dropdown.currentIndexChanged.connect(self.update_data)

        self.ra_label = QLabel("Right Ascension: ")
        self.dec_label = QLabel("Declination: ")
        self.dist_label = QLabel("Distance (AU): ")

        self.figure = Figure(figsize=(6,6))
        self.canvas = FigureCanvas(self.figure)
        self.ax = self.figure.add_subplot(111, projection='3d')

        layout.addWidget(self.dropdown)
        layout.addWidget(self.ra_label)
        layout.addWidget(self.dec_label)
        layout.addWidget(self.dist_label)
        layout.addWidget(self.canvas)

        self.setLayout(layout)

        self.timer = QTimer()
        self.timer.timeout.connect(self.update_data)
        self.timer.start(1000)

        self.update_data()

    def update_data(self):
        selected = self.dropdown.currentText()
        t = ts.now()
        sun = planets['sun']

        self.ax.clear()
        self.ax.set_title("3D Positions Relative to Sun (AU, scaled)")
        self.ax.set_xlabel('X (AU)')
        self.ax.set_ylabel('Y (AU)')
        self.ax.set_zlabel('Z (AU)')
        self.ax.grid(True)

        # Sun at origin
        self.ax.scatter(0, 0, 0, color='yellow', s=200, label='Sun', edgecolors='orange', linewidths=1)

        if selected == 'All Planets':
            max_range = 0
            for name, sf_name in planet_names.items():
                if sf_name == 'sun':
                    continue  # already plotted Sun
                target = planets[sf_name]
                astrometric = sun.at(t).observe(target)
                pos = astrometric.position.au
                scaled_pos = scale_position(pos)
                self.ax.scatter(*scaled_pos, s=80, label=name)
                max_range = max(max_range, *(abs(c) for c in scaled_pos))

            max_range *= 1.2
            self.ax.set_xlim(-max_range, max_range)
            self.ax.set_ylim(-max_range, max_range)
            self.ax.set_zlim(-max_range, max_range)

            self.ra_label.setText("Right Ascension: N/A")
            self.dec_label.setText("Declination: N/A")
            self.dist_label.setText("Distance (AU): N/A")

        else:
            if selected == 'Sun':
                # Sun is at origin, no RA/Dec/distance info needed
                self.ra_label.setText("Right Ascension: N/A")
                self.dec_label.setText("Declination: N/A")
                self.dist_label.setText("Distance (AU): 0.0000")
                self.ax.set_xlim(-1,1)
                self.ax.set_ylim(-1,1)
                self.ax.set_zlim(-1,1)
            else:
                skyfield_name = planet_names[selected]
                target = planets[skyfield_name]
                astrometric = sun.at(t).observe(target)
                ra, dec, distance = astrometric.radec()
                pos = astrometric.position.au
                scaled_pos = scale_position(pos)

                self.ax.scatter(*scaled_pos, color='red', s=100, label=selected)

                max_range = max(abs(c) for c in scaled_pos) * 1.2
                max_range = max(max_range, 1)
                self.ax.set_xlim(-max_range, max_range)
                self.ax.set_ylim(-max_range, max_range)
                self.ax.set_zlim(-max_range, max_range)

                self.ra_label.setText(f"Right Ascension: {ra}")
                self.dec_label.setText(f"Declination: {dec}")
                self.dist_label.setText(f"Distance (AU): {distance.au:.4f}")

        self.ax.legend()
        self.canvas.draw()

if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = PlanetTracker3D()
    window.show()
    sys.exit(app.exec_())



In [ ]:
import sys
from PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout, QLabel, QComboBox
from PyQt5.QtCore import QTimer
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.figure import Figure
from mpl_toolkits.mplot3d import Axes3D
from skyfield.api import load
import numpy as np

# Load planetary data
planets = load('de421.bsp')
ts = load.timescale()

planet_names = {
    'Sun': 'sun',
    'Mercury': 'mercury',
    'Venus': 'venus',
    'Earth': 'earth',
    'Mars': 'mars',
    'Jupiter': 'jupiter barycenter',
    'Saturn': 'saturn barycenter',
    'Uranus': 'uranus barycenter',
    'Neptune': 'neptune barycenter',
    'Pluto': 'pluto barycenter'
}

class PlanetTracker3DRealistic(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("3D Realistic Scale Planet Tracker with Sun")
        self.setGeometry(100, 100, 700, 700)

        layout = QVBoxLayout()

        self.dropdown = QComboBox()
        self.dropdown.addItems(['All Planets'] + list(planet_names.keys()))
        self.dropdown.currentIndexChanged.connect(self.update_data)

        self.ra_label = QLabel("Right Ascension: ")
        self.dec_label = QLabel("Declination: ")
        self.dist_label = QLabel("Distance (AU): ")

        self.figure = Figure(figsize=(6,6))
        self.canvas = FigureCanvas(self.figure)
        self.ax = self.figure.add_subplot(111, projection='3d')

        layout.addWidget(self.dropdown)
        layout.addWidget(self.ra_label)
        layout.addWidget(self.dec_label)
        layout.addWidget(self.dist_label)
        layout.addWidget(self.canvas)

        self.setLayout(layout)

        self.timer = QTimer()
        self.timer.timeout.connect(self.update_data)
        self.timer.start(1000)

        self.update_data()

    def update_data(self):
        selected = self.dropdown.currentText()
        t = ts.now()
        sun = planets['sun']

        self.ax.clear()
        self.ax.set_title("3D Positions Relative to Sun (True AU Scale)")
        self.ax.set_xlabel('X (AU)')
        self.ax.set_ylabel('Y (AU)')
        self.ax.set_zlabel('Z (AU)')
        self.ax.grid(True)

        # Sun at origin
        self.ax.scatter(0, 0, 0, color='yellow', s=300, label='Sun', edgecolors='orange', linewidths=1)

        if selected == 'All Planets':
            max_range = 0
            for name, sf_name in planet_names.items():
                if sf_name == 'sun':
                    continue  # already plotted Sun
                target = planets[sf_name]
                astrometric = sun.at(t).observe(target)
                pos = astrometric.position.au
                self.ax.scatter(*pos, s=80, label=name)
                max_range = max(max_range, *(abs(c) for c in pos))

            max_range *= 1.1  # little margin
            self.ax.set_xlim(-max_range, max_range)
            self.ax.set_ylim(-max_range, max_range)
            self.ax.set_zlim(-max_range, max_range)

            self.ra_label.setText("Right Ascension: N/A")
            self.dec_label.setText("Declination: N/A")
            self.dist_label.setText("Distance (AU): N/A")

        else:
            if selected == 'Sun':
                self.ra_label.setText("Right Ascension: N/A")
                self.dec_label.setText("Declination: N/A")
                self.dist_label.setText("Distance (AU): 0.0000")
                self.ax.set_xlim(-1, 1)
                self.ax.set_ylim(-1, 1)
                self.ax.set_zlim(-1, 1)
            else:
                skyfield_name = planet_names[selected]
                target = planets[skyfield_name]
                astrometric = sun.at(t).observe(target)
                ra, dec, distance = astrometric.radec()
                pos = astrometric.position.au
                self.ax.scatter(*pos, color='red', s=120, label=selected)

                max_range = max(abs(c) for c in pos) * 1.1
                max_range = max(max_range, 1)
                self.ax.set_xlim(-max_range, max_range)
                self.ax.set_ylim(-max_range, max_range)
                self.ax.set_zlim(-max_range, max_range)

                self.ra_label.setText(f"Right Ascension: {ra}")
                self.dec_label.setText(f"Declination: {dec}")
                self.dist_label.setText(f"Distance (AU): {distance.au:.4f}")

        self.ax.legend()
        self.canvas.draw()

if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = PlanetTracker3DRealistic()
    window.show()
    sys.exit(app.exec_())


In [ ]:
import sys
from PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout, QLabel, QComboBox
from PyQt5.QtCore import QTimer
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.figure import Figure
from mpl_toolkits.mplot3d import Axes3D
from skyfield.api import load
import numpy as np

# Load planetary data
planets = load('de421.bsp')
ts = load.timescale()

planet_names = {
    'Sun': 'sun',
    'Mercury': 'mercury',
    'Venus': 'venus',
    'Earth': 'earth',
    'Mars': 'mars',
    'Jupiter': 'jupiter barycenter',
    'Saturn': 'saturn barycenter',
    'Uranus': 'uranus barycenter',
    'Neptune': 'neptune barycenter',
    'Pluto': 'pluto barycenter'
}

# Approximate planet radii in Earth radii (used for marker size)
planet_radii = {
    'Sun': 109,
    'Mercury': 0.38,
    'Venus': 0.95,
    'Earth': 1.0,
    'Mars': 0.53,
    'Jupiter': 11.2,
    'Saturn': 9.45,
    'Uranus': 4.0,
    'Neptune': 3.88,
    'Pluto': 0.18
}

class PlanetTracker3DRealistic(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("3D Realistic Planet Tracker (True Scale with Sun)")
        self.setGeometry(100, 100, 1200, 900)

        layout = QVBoxLayout()

        self.dropdown = QComboBox()
        self.dropdown.addItems(['All Planets'] + list(planet_names.keys()))
        self.dropdown.currentIndexChanged.connect(self.update_data)

        self.ra_label = QLabel("Right Ascension: ")
        self.dec_label = QLabel("Declination: ")
        self.dist_label = QLabel("Distance (AU): ")

        self.figure = Figure(figsize=(12, 10))
        self.canvas = FigureCanvas(self.figure)
        self.ax = self.figure.add_subplot(111, projection='3d')

        layout.addWidget(self.dropdown)
        layout.addWidget(self.ra_label)
        layout.addWidget(self.dec_label)
        layout.addWidget(self.dist_label)
        layout.addWidget(self.canvas)

        self.setLayout(layout)

        self.timer = QTimer()
        self.timer.timeout.connect(self.update_data)
        self.timer.start(1000)

        self.update_data()

    def update_data(self):
        selected = self.dropdown.currentText()
        t = ts.now()
        sun = planets['sun']

        self.ax.clear()
        self.ax.set_title("Solar System 3D View (True AU Distances)")
        self.ax.set_xlabel('X (AU)')
        self.ax.set_ylabel('Y (AU)')
        self.ax.set_zlabel('Z (AU)')
        self.ax.grid(True)

        # Plot Sun at origin
        sun_size = planet_radii['Sun'] * 5
        self.ax.scatter(0, 0, 0, color='yellow', s=sun_size, label='Sun', edgecolors='orange', linewidths=1)
        self.ax.text(0.2, 0.2, 0.2, 'Sun', fontsize=9)

        if selected == 'All Planets':
            max_range = 0
            for name, sf_name in planet_names.items():
                if sf_name == 'sun':
                    continue
                target = planets[sf_name]
                astrometric = sun.at(t).observe(target)
                pos = astrometric.position.au
                size = planet_radii.get(name, 1) * 5
                self.ax.scatter(*pos, s=size, label=name)
                self.ax.text(pos[0], pos[1], pos[2], name, fontsize=8)
                max_range = max(max_range, np.linalg.norm(pos))

            # Set axis limits to accommodate Neptune (~30 AU)
            max_range = max(max_range, 30) * 1.1
            self.ax.set_xlim(-max_range, max_range)
            self.ax.set_ylim(-max_range, max_range)
            self.ax.set_zlim(-max_range, max_range)

            self.ra_label.setText("Right Ascension: N/A")
            self.dec_label.setText("Declination: N/A")
            self.dist_label.setText("Distance (AU): N/A")

        else:
            if selected == 'Sun':
                self.ra_label.setText("Right Ascension: N/A")
                self.dec_label.setText("Declination: N/A")
                self.dist_label.setText("Distance (AU): 0.0000")
                self.ax.set_xlim(-1, 1)
                self.ax.set_ylim(-1, 1)
                self.ax.set_zlim(-1, 1)
            else:
                target = planets[planet_names[selected]]
                astrometric = sun.at(t).observe(target)
                ra, dec, distance = astrometric.radec()
                pos = astrometric.position.au
                size = planet_radii.get(selected, 1) * 5
                self.ax.scatter(*pos, s=size, color='red', label=selected)
                self.ax.text(pos[0], pos[1], pos[2], selected, fontsize=10)

                max_range = max(abs(c) for c in pos) * 1.2
                max_range = max(max_range, 1)
                self.ax.set_xlim(-max_range, max_range)
                self.ax.set_ylim(-max_range, max_range)
                self.ax.set_zlim(-max_range, max_range)

                self.ra_label.setText(f"Right Ascension: {ra}")
                self.dec_label.setText(f"Declination: {dec}")
                self.dist_label.setText(f"Distance (AU): {distance.au:.4f}")

        # Place legend far outside the plot (to the right)
        self.ax.legend(loc='center left', bbox_to_anchor=(1.25, 0.5), fontsize=8)
        self.canvas.draw()

if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = PlanetTracker3DRealistic()
    window.show()
    sys.exit(app.exec_())


In [ ]:
import sys
from PyQt5.QtWidgets import (
    QApplication, QWidget, QVBoxLayout, QLabel, QComboBox, QPushButton, QHBoxLayout
)
from PyQt5.QtCore import QTimer
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.figure import Figure
from mpl_toolkits.mplot3d import Axes3D
from skyfield.api import load
import numpy as np

# Load planetary data
planets = load('de421.bsp')
ts = load.timescale()

planet_names = {
    'Sun': 'sun',
    'Mercury': 'mercury',
    'Venus': 'venus',
    'Earth': 'earth',
    'Mars': 'mars',
    'Jupiter': 'jupiter barycenter',
    'Saturn': 'saturn barycenter',
    'Uranus': 'uranus barycenter',
    'Neptune': 'neptune barycenter',
    'Pluto': 'pluto barycenter'
}

# Approximate planet radii in Earth radii (used for marker size)
planet_radii = {
    'Sun': 109,
    'Mercury': 0.38,
    'Venus': 0.95,
    'Earth': 1.0,
    'Mars': 0.53,
    'Jupiter': 11.2,
    'Saturn': 9.45,
    'Uranus': 4.0,
    'Neptune': 3.88,
    'Pluto': 0.18
}

class PlanetTracker3DRealistic(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("3D Realistic Planet Tracker (True Scale with Sun)")
        self.setGeometry(100, 100, 1200, 900)

        layout = QVBoxLayout()

        self.dropdown = QComboBox()
        self.dropdown.addItems(['All Planets'] + list(planet_names.keys()))
        self.dropdown.currentIndexChanged.connect(self.update_data)

        self.ra_label = QLabel("Right Ascension: ")
        self.dec_label = QLabel("Declination: ")
        self.dist_label = QLabel("Distance (AU): ")

        self.figure = Figure(figsize=(12, 10))
        self.canvas = FigureCanvas(self.figure)
        self.ax = self.figure.add_subplot(111, projection='3d')

        layout.addWidget(self.dropdown)
        layout.addWidget(self.ra_label)
        layout.addWidget(self.dec_label)
        layout.addWidget(self.dist_label)
        layout.addWidget(self.canvas)
        self.setLayout(layout)

        self.timer = QTimer()
        self.timer.timeout.connect(self.update_data)
        self.timer.start(1000)

        # Connect scroll event for zooming
        self.canvas.mpl_connect('scroll_event', self.on_scroll)

        self._zoom_scale = 1.0
        self._base_limit = 35  # Initial full range

        self.update_data()

    def on_scroll(self, event):
        # Zoom factor (adjust as desired)
        scale_factor = 0.9 if event.button == 'up' else 1.1
        self._zoom_scale *= scale_factor

        new_limit = self._base_limit * self._zoom_scale
        self.ax.set_xlim(-new_limit, new_limit)
        self.ax.set_ylim(-new_limit, new_limit)
        self.ax.set_zlim(-new_limit, new_limit)
        self.canvas.draw()

    def update_data(self):
        import matplotlib.pyplot as plt

        selected = self.dropdown.currentText()
        t = ts.now()
        sun = planets['sun']

        self.ax.clear()
        self.ax.set_title("Solar System 3D View (True AU Distances)")
        self.ax.set_xlabel('X (AU)')
        self.ax.set_ylabel('Y (AU)')
        self.ax.set_zlabel('Z (AU)')
        self.ax.grid(True)

        sun_size = planet_radii['Sun'] * 5
        self.ax.scatter(0, 0, 0, color='yellow', s=sun_size, label='Sun', edgecolors='orange', linewidths=1)
        self.ax.text(0.2, 0.2, 0.2, 'Sun', fontsize=9)

        if selected == 'All Planets':
            max_range = 0
            for idx, (name, sf_name) in enumerate(planet_names.items()):
                if sf_name == 'sun':
                    continue

                target = planets[sf_name]
                astrometric = sun.at(t).observe(target)
                pos = astrometric.position.au
                size = planet_radii.get(name, 1) * 5
                color = plt.cm.tab10(idx % 10)

                self.ax.scatter(*pos, s=size, label=name, color=color)
                self.ax.text(pos[0], pos[1], pos[2], name, fontsize=8)

                # Plot orbital path
                orbit_times = ts.linspace(t - 182.5, t + 182.5, 200)  # ~1 Earth year
                orbit_positions = sun.at(orbit_times).observe(target).position.au
                self.ax.plot(orbit_positions[0], orbit_positions[1], orbit_positions[2], color=color, linewidth=0.8, alpha=0.6)

                max_range = max(max_range, np.max(np.linalg.norm(orbit_positions, axis=0)))

            max_range = max(max_range, 30) * 1.1
            self._base_limit = max_range
            limit = self._base_limit * self._zoom_scale
            self.ax.set_xlim(-limit, limit)
            self.ax.set_ylim(-limit, limit)
            self.ax.set_zlim(-limit, limit)

            self.ra_label.setText("Right Ascension: N/A")
            self.dec_label.setText("Declination: N/A")
            self.dist_label.setText("Distance (AU): N/A")

        else:
            if selected == 'Sun':
                self.ra_label.setText("Right Ascension: N/A")
                self.dec_label.setText("Declination: N/A")
                self.dist_label.setText("Distance (AU): 0.0000")
                self.ax.set_xlim(-1, 1)
                self.ax.set_ylim(-1, 1)
                self.ax.set_zlim(-1, 1)
            else:
                target = planets[planet_names[selected]]
                astrometric = sun.at(t).observe(target)
                ra, dec, distance = astrometric.radec()
                pos = astrometric.position.au
                size = planet_radii.get(selected, 1) * 5
                self.ax.scatter(*pos, s=size, color='red', label=selected)
                self.ax.text(pos[0], pos[1], pos[2], selected, fontsize=10)

                # Optional: Show orbit of selected planet
                orbit_times = ts.linspace(t - 182.5, t + 182.5, 200)
                orbit_positions = sun.at(orbit_times).observe(target).position.au
                self.ax.plot(orbit_positions[0], orbit_positions[1], orbit_positions[2], color='red', linewidth=0.8, alpha=0.6)

                max_range = max(abs(c) for c in pos) * 1.2
                max_range = max(max_range, 1)
                self._base_limit = max_range
                limit = self._base_limit * self._zoom_scale
                self.ax.set_xlim(-limit, limit)
                self.ax.set_ylim(-limit, limit)
                self.ax.set_zlim(-limit, limit)

                self.ra_label.setText(f"Right Ascension: {ra}")
                self.dec_label.setText(f"Declination: {dec}")
                self.dist_label.setText(f"Distance (AU): {distance.au:.4f}")

        self.ax.legend(loc='center left', bbox_to_anchor=(1.25, 0.5), fontsize=8)
        self.canvas.draw()



if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = PlanetTracker3DRealistic()
    window.show()
    sys.exit(app.exec_())


In [1]:
import sys
from PyQt5.QtWidgets import (
    QApplication, QWidget, QVBoxLayout, QLabel, QComboBox
)
from PyQt5.QtCore import QTimer
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.figure import Figure
from mpl_toolkits.mplot3d import Axes3D
from skyfield.api import load
import numpy as np
import matplotlib.pyplot as plt

planets = load('de421.bsp')
ts = load.timescale()

planet_names = {
    'Sun': 'sun',
    'Mercury': 'mercury',
    'Venus': 'venus',
    'Earth': 'earth',
    'Mars': 'mars',
    'Jupiter': 'jupiter barycenter',
    'Saturn': 'saturn barycenter',
    'Uranus': 'uranus barycenter',
    'Neptune': 'neptune barycenter',
    'Pluto': 'pluto barycenter'
}

# Approximate planet radii in Earth radii (for marker size)
planet_radii = {
    'Sun': 109,
    'Mercury': 0.38,
    'Venus': 0.95,
    'Earth': 1.0,
    'Mars': 0.53,
    'Jupiter': 11.2,
    'Saturn': 9.45,
    'Uranus': 4.0,
    'Neptune': 3.88,
    'Pluto': 0.18
}

# Orbital periods in days for each planet (approximate)
orbital_periods_days = {
    'Mercury': 87.969,
    'Venus': 224.701,
    'Earth': 365.256,
    'Mars': 686.980,
    'Jupiter': 4332.59,
    'Saturn': 10759.22,
    'Uranus': 30688.5,
    'Neptune': 60182,
    'Pluto': 90560
}

class PlanetTracker3DFullOrbit(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("3D Planet Tracker with Full Orbital Paths")
        self.setGeometry(100, 100, 1200, 900)

        layout = QVBoxLayout()

        self.dropdown = QComboBox()
        self.dropdown.addItems(['All Planets'] + list(planet_names.keys()))
        self.dropdown.currentIndexChanged.connect(self.update_data)

        self.ra_label = QLabel("Right Ascension: ")
        self.dec_label = QLabel("Declination: ")
        self.dist_label = QLabel("Distance (AU): ")

        self.figure = Figure(figsize=(12, 10))
        self.canvas = FigureCanvas(self.figure)
        self.ax = self.figure.add_subplot(111, projection='3d')

        layout.addWidget(self.dropdown)
        layout.addWidget(self.ra_label)
        layout.addWidget(self.dec_label)
        layout.addWidget(self.dist_label)
        layout.addWidget(self.canvas)
        self.setLayout(layout)

        self.timer = QTimer()
        self.timer.timeout.connect(self.update_data)
        self.timer.start(1000)

        self.canvas.mpl_connect('scroll_event', self.on_scroll)

        self._zoom_scale = 1.0
        self._base_limit = 35

        self.update_data()

    def on_scroll(self, event):
        scale_factor = 0.9 if event.button == 'up' else 1.1
        self._zoom_scale *= scale_factor

        new_limit = self._base_limit * self._zoom_scale
        self.ax.set_xlim(-new_limit, new_limit)
        self.ax.set_ylim(-new_limit, new_limit)
        self.ax.set_zlim(-new_limit, new_limit)
        self.canvas.draw()

    def plot_full_orbit(self, target, name, color):
        sun = planets['sun']

        # Ephemeris valid time range
        ephem_start = ts.utc(1899, 7, 29)
        ephem_end = ts.utc(2053, 10, 9)

        if name == 'Sun':
            return  # no orbit for the Sun

        # Start date for orbit - keep inside ephemeris range
        start_time = ts.utc(2000, 1, 1)
        if start_time < ephem_start:
            start_time = ephem_start
        if start_time > ephem_end:
            start_time = ephem_end

        days = orbital_periods_days.get(name, 365.25)
        end_time_candidate = start_time + days

        # Clip to ephemeris range
        if end_time_candidate > ephem_end:
            end_time = ephem_end
        else:
            end_time = end_time_candidate

        # Number of days we can plot (might be less than full orbit)
        plot_days = end_time - start_time  # This is already a float number of days


        # If plot_days too small, fallback to 1 year
        if plot_days < 50:
            start_time = ephem_start
            end_time = ephem_start + 365.25
            plot_days = 365.25

        times = ts.linspace(start_time, end_time, 500)

        positions = sun.at(times).observe(target).position.au

        self.ax.plot(positions[0], positions[1], positions[2],
                    color=color, linewidth=0.8, alpha=0.6)

        # current position at now()
        t_now = ts.now()
        current_pos = sun.at(t_now).observe(target).position.au
        size = planet_radii.get(name, 1) * 5
        self.ax.scatter(*current_pos, s=size, color=color, label=name)
        self.ax.text(current_pos[0], current_pos[1], current_pos[2], name, fontsize=8)

        return positions


    def update_data(self):
        selected = self.dropdown.currentText()
        sun = planets['sun']

        self.ax.clear()
        self.ax.set_title("Solar System 3D View with Full Orbital Paths (AU)")
        self.ax.set_xlabel('X (AU)')
        self.ax.set_ylabel('Y (AU)')
        self.ax.set_zlabel('Z (AU)')
        self.ax.grid(True)

        sun_size = planet_radii['Sun'] * 5
        self.ax.scatter(0, 0, 0, color='yellow', s=sun_size,
                        label='Sun', edgecolors='orange', linewidths=1)
        self.ax.text(0.2, 0.2, 0.2, 'Sun', fontsize=9)

        if selected == 'All Planets':
            max_range = 0
            for idx, (name, sf_name) in enumerate(planet_names.items()):
                if sf_name == 'sun':
                    continue
                target = planets[sf_name]
                color = plt.cm.tab10(idx % 10)
                orbit_positions = self.plot_full_orbit(target, name, color)
                if orbit_positions is not None:
                    max_range = max(max_range, np.max(np.linalg.norm(orbit_positions, axis=0)))

            max_range = max(max_range, 30) * 1.1
            self._base_limit = max_range
            limit = self._base_limit * self._zoom_scale
            self.ax.set_xlim(-limit, limit)
            self.ax.set_ylim(-limit, limit)
            self.ax.set_zlim(-limit, limit)

            self.ra_label.setText("Right Ascension: N/A")
            self.dec_label.setText("Declination: N/A")
            self.dist_label.setText("Distance (AU): N/A")

        else:
            if selected == 'Sun':
                self.ra_label.setText("Right Ascension: N/A")
                self.dec_label.setText("Declination: N/A")
                self.dist_label.setText("Distance (AU): 0.0000")
                self.ax.set_xlim(-1, 1)
                self.ax.set_ylim(-1, 1)
                self.ax.set_zlim(-1, 1)
            else:
                target = planets[planet_names[selected]]
                astrometric = sun.at(ts.now()).observe(target)
                ra, dec, distance = astrometric.radec()
                pos = astrometric.position.au
                size = planet_radii.get(selected, 1) * 5
                color = 'red'
                self.ax.scatter(*pos, s=size, color=color, label=selected)
                self.ax.text(pos[0], pos[1], pos[2], selected, fontsize=10)

                self.plot_full_orbit(target, selected, color)

                max_range = max(abs(c) for c in pos) * 1.2
                max_range = max(max_range, 1)
                self._base_limit = max_range
                limit = self._base_limit * self._zoom_scale
                self.ax.set_xlim(-limit, limit)
                self.ax.set_ylim(-limit, limit)
                self.ax.set_zlim(-limit, limit)

                self.ra_label.setText(f"Right Ascension: {ra}")
                self.dec_label.setText(f"Declination: {dec}")
                self.dist_label.setText(f"Distance (AU): {distance.au:.4f}")

        self.ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
        self.canvas.draw()

if __name__ == "__main__":
    app = QApplication(sys.argv)
    tracker = PlanetTracker3DFullOrbit()
    tracker.show()
    sys.exit(app.exec_())


SystemExit: 0

c:\Users\Juan Dominguez\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
